Author: Weiwen Leung

Date created: October 2018

Purpose: To create a contextual multi-armed bandit that uses Bayesian Linear Regression and Thompson Sampling.

Acknowledgements: Linear Regression equations taken from Bishop (2006)

# What is a Contextual Bandit?

A multi-armed bandit is an algorithm that self-learns the optimal treatment to achieve an outcome.

A **contextual** multi-armed bandit is an algorithm that self-learns the optimal **personalized** treatment to achieve an outcome. (In other words, a contextual bandit is a generalization of the multi-armed bandit)

Consider the case of encouraging people to donate to charity. Imagine you want to encourage people to donate to charity, and you have five different messages to encourage charitable giving, but are not sure which works best. A multi-armed bandit would generally initially randomly test explanations. However, as users see the explanations and make their donation decisions, more effective messages are more likely to be shown more frequently. For example, suppose that upon seeing a certain message, a user makes a large donation. That message will be slightly more likely to be shown to the next user.

A **contextual** multi-armed bandit does everything that a multi-armed bandit does, but also takes into account user characteristics (e.g. age, gender, location). For example, the donations of male users impact the probability of messages that will be displayed to male users more than the probability of messages that will be displayed to female users. This can be very useful for providing personalized treatments; e.g. males may be more motivated to donate if the donation appeal contains a photo of a female, while females may be more motivated to donate if the donation appeal contains a photo of a male.

# Structure of this notebook

There are two classes: 

1. BayesLinear: creates Bayesian Linear Regression objects that allow for personalized treatments

2. Experiment: runs experiments that use Bayesian Linear Regressions to provide personalized treatments

To create a contextual bandit, first create a BayesLinear object (specifying the priors) 

Then create an Experiment object (specify the number of covariates, the number of experimental variables, a BayesLinear object, the data source, and the data generating process).


# "Experiment" Flow

      1. load observation (participant characteristics)
      2. conduct Thompson Sampling (take a random sample from distribution of coefficients)
      3. calculate which treatments to include (i.e. which treatments increase donations, if donations is the dependent variable)
      4. give treatments, observe reward (reward calculated from data generating process)
      5. calculate posterior by Bayesian updating.
      6. when next observation comes, posterior becomes new prior.
      
# Mechanics of Bayesian Linear Regression

## Notation:
$t$: target (statisticians call this the "dependent variable")

$\vec{w}$: weight vector ("regression coefficients")

$\vec{x}$: raw feature vector ("raw data")

$\phi$: transformed feature vector ("data")

$\beta$: precision of noise 

 $\epsilon$: error term/residual
 
$\vec{m}_0, \vec{m}_N$: prior of mean of weight vector after $0$ and $N$ observations respectively.

$\vec{S}_0, \vec{S}_N$: prior of covariance of weight vector after $0$ and $N$ observations respectively.

## The Target
$$t = f(\vec{x}, \vec{w}) + \epsilon $$


### The Model

$$p(t \mid \vec{x}, \vec{w}) =  Norm(t \mid y(\vec{x}, \vec{w}), \beta^{-1}) = Norm(t \mid \vec{w}^T \vec{\phi}(\vec{x}), \beta^{-1}) $$ 

hope: to find values for $\vec{w}$ that make a good fit to the true model, given by 


### Theory

$$p(\vec{w}) = Norm(\vec{w} \mid \vec{m}_0, \vec{S}_0) $$


where the parameters are updated via these equations:
$$ \vec{S}^{-1}_N = \vec{S}_0^{-1} + \beta\vec{\Phi}^T\vec{\Phi} $$
$$ \vec{m}_N = \vec{S}_N(\vec{S}_0^{-1}\vec{m}_0 + \beta\vec{\Phi}^T\vec{t}) $$

Note that if data comes in sequentially, the posterior of the previous step becomes the prior of the current step and we only need to calculate the updates to $\vec{m}_N$ and $\vec{S}_N^{-1}$.




In [0]:
import numpy as np
from numpy.random import normal, uniform
from scipy.stats import multivariate_normal as mv_norm
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [0]:
class LinearBayes(object):
    """
    A class that holds parameter prior/posterior and handles 
    the hyper-parameter updates with new data
    
    Note:  variables starting with "v_" indicate Nx1 dimensional 
        column vectors, those starting with "m_" indicate 
        matrices, and those starting with "a_" indicate 
        1xN dimensional arrays.
    
    Args:
        a_m0 (np.array): prior mean vector of size 1xM
        m_S0 (np.ndarray): prior covariance matrix of size MxM
        beta (float): known real-data noise precision
        
    """
    def __init__(self, a_m0, m_S0, beta):
        self.prior = mv_norm(mean=a_m0, cov=m_S0)
        self.v_m0 = a_m0.reshape(a_m0.shape + (1,)) #reshape to column vector
        self.m_S0 = m_S0
        self.beta = beta
        
        self.v_mN = self.v_m0
        self.m_SN = self.m_S0
        self.posterior = self.prior
           
    def get_phi(self, a_x):
        """
        Returns the design matrix of size (NxM) for a feature vector v_x.
        In this case, this function merely adds the phi_0 dummy basis
        that's equal to 1 for all elements.
        
        Args:
            a_x (np.array): input features of size 1xN
        """
        if len(a_x.shape) > 1: # if there is more than one observation per batch
          m_phi = np.ones(a_x.shape[0], a_x.shape[1] + 1)
          m_phi[:, 1:a_x.shape[1]] = a_x
        else: # if there is exactly one observation per batch
          #m_phi = np.append([1], a_x) # no need to append [1]. Weiwen commented because it seems that you are adding a constant twice
          m_phi = a_x
          m_phi = np.array([m_phi])

        
        return m_phi
        
    def set_posterior(self, a_x, a_t):
        """
        Updates mN and SN given vectors of x-values and t-values
        """
        # Need to convert v_t from an array into a column vector
        # to correctly compute matrix multiplication
        v_t = a_t.reshape(a_t.shape + (1,))

        m_phi = self.get_phi(a_x)
        self.m_SN = np.linalg.inv(np.linalg.inv(self.m_S0) + self.beta*m_phi.T.dot(m_phi))
        if len(v_t.shape) == 1:
          v_t = np.array([v_t]) # casts a 1D array as a 2D in order to avoid adding (M, 1) array to an (M,) array and getting (M by M)
        self.v_mN = self.m_SN.dot(np.linalg.inv(self.m_S0).dot(self.v_m0) + \
                                      self.beta*m_phi.T.dot(v_t))
        
        self.posterior = mv_norm(mean=self.v_mN.flatten(), cov=self.m_SN)

    
    def thompson_sampling(self):
      """
      Performs Thompson Sampling on the prior distributions of regression coefficients.
      """      
      return np.random.multivariate_normal(self.v_m0.flatten(), self.m_S0) # note np multivariate_normal draws a sample, while scipy's multivariate normal is the random variable itself
    
    
    def replace_prior(self):
      """
      After the posterior has been updated, the current prior becomes the posterior for the next data point(s)
      """
      self.prior = self.posterior
      self.v_m0 = self.prior.mean.reshape(self.prior.mean.shape + (1,)) #reshape to column vector
      self.m_S0 = self.prior.cov
      
    


After you create a BayesLinear object, create an Experiment object that takes as input:

* number of covariates
* number of experimental variables
*   BayesLinear object
*   data set (created through the user-specified data generating process; in future, will allow CSV)
* regression coefficients for data generating process

By convention, order of regression coefficients is as follows:

$[constant, covar1, covar2, ..., covarM, constant*exptvar1, covar1*exptvar1, ..., covarM*exptvar1, constant*exptvar2, covar1*exptvar2, ..., constant*exptvar3, ...]$


For example, with one covariate and two experimental treatments, we have:

$[constant, covar1, exptvar1, exptvar1*covar1, exptvar2, exptvar2*covar1]$

i.e. regression equation can be written as $y = \beta_0 + \beta_1*cov1 + \beta_2*expt1 + \beta_3*expt1*cov1 + \beta_4*expt2 + \beta_5*expt2*cov1$

Note that currently, we only interact experimental treatments with covariates. 

The code written allows covariates to take on any (real) value, but experimental treatments are restricted to binary.

In [0]:
class Experiment:
  """
  A class that contains details of each experiment that is run.
  
  Parameters
  ==========
  
  Covariates: number of covariates (int)
  Experimental Variables: number of experimental variables (int)
  Regression Model: LinearBayes object  
  
  This class uses the convention that in the dataset, covariates always appear to the LEFT of experimental variables.
  """
  def __init__(self, covariates, exptvar, regressionmodel, datasource, coeffs):
    self.covariates = covariates
    self.exptvar = exptvar
    self.regressionmodel = regressionmodel
    self.data = datasource #np.loadtxt(datasource, delimiter = ",", skiprows = 1)
    self.numobs = len(self.data)
    self.coeffs = coeffs
      
    
  #def calculate_expected_value_in_each_treatment(self, observation, experiment):
  def calculate_which_treatments_to_include(self, observation):
    """
    Use the convention in the dataset:
    [constant, covar1, covar2, ..., covarM, constant*exptvar1, covar1*exptvar1, ..., covarM*exptvar1, constant*exptvar2, covar1*exptvar2, ..., constant*exptvar3, ...]
    assume each treatment is binary and treatments are mutually exclusive.

    Pseudo-code:
    For each experimental variable, calculate the contribution to expected value of dep var. Determine whether or not it should be included 
    (threshold: sum of all terms including exptvar1 is above zero, justified under the assumption that treatments do not interact).
    """
    #exptvar = experiment.exptvar
    #covariates = experiment.covariates 
    #numinteractions = exptvar*covariates      

    coefficients = self.regressionmodel.thompson_sampling()
    observationplusconstant = np.append(np.array([1]), observation)
    treatments_to_use = []
    for i in range(self.exptvar):
      expectedeffect = 0
      for j in range(self.covariates + 1):
        # NOTE: add 1 to covariates because need constant
        # Note we omit covariates as they are fixed.

        expectedeffect += coefficients[(i + 1)*(self.covariates + 1) + j]*observationplusconstant[j]
        # How this works in the case of 2 covar and 2 expt var: 
        # [constant cov1 cov2 expt1 expt1*cov1 expt1*cov2 expt2 expt2*cov1 expt2*cov2 expt3 expt3*cov1 expt3*cov2]
        # 3 4 5, ... , 6, 7, 8, ... 9, 10, 11

      if expectedeffect > (10 ** -5): # allow for floating point error
        treatments_to_use += [i]

    return treatments_to_use

  def observe_reward(self, observationi, chosentreatments):
    """
    Calculates reward based on observed characteristics and chosen treatments.
    Real reward function has been specified by user before conducting simulations
    """
    numcoeffs = (1 + len(observationi))*(1 + self.exptvar) # assuming constant is not included in observationi, else change to len(observationi)    
    chosentreatments = [i + 1 for i in chosentreatments]
    chosentreatments = [0] + chosentreatments # include intercept in chosen treatments; makes sure that initial values are not set to zero
    observationi = np.append([1], observationi) # adds constant
    values = np.array([observationi[j] if i in chosentreatments else 0 for i in range(self.exptvar + 1) for j in range(self.covariates + 1) ]) # assumes constant is included in observationi
    # [constant cov1 cov2 expt1 expt1*cov1 expt1*cov2 expt2 expt2*cov1 expt2*cov2 expt3 expt3*cov1 expt3*cov2]
    reward = np.dot(self.coeffs, values) + np.random.normal(scale = 3.0)
    #print(observationi, chosentreatments, values, reward)
    return reward, values
  
  def run_experiment(self, numpulls):
    """
    Loads dataset
    
    For each observation:
      Load observation (participant characteristics)
      Thompson Sampling
      calculate_which_treatments_to_include
      Give treatments, observe reward
      update prior      
    """
    treatmentsgiven = np.zeros((self.exptvar,self.numobs))
    # Expand data matrix to allow for interactions
    
    for i in range(numpulls):
      observationi = self.data[i] # observationi only has covariates, i.e. no constants, no interactions between exptvar and covar
      chosentreatments = self.calculate_which_treatments_to_include(observationi) # the line which calls Thompson Sampling implicitly assumes that model has covariates and a constant. INITIALIZE BAYESLINEAR OBJECT ACCORDINGLY
      reward, values = self.observe_reward(observationi, chosentreatments) # seems ok, observationi assumed NOT to include covariates or interactions. "values" includes constant and all covariates-exptvar interactions
      self.regressionmodel.set_posterior(values, reward) 
      self.regressionmodel.replace_prior()
      #print(self.regressionmodel.v_mN[2:6])
      #print(observationi, chosentreatments)
      for j in chosentreatments:
        # record treatments
        treatmentsgiven[j,i] = 1
        
    return treatmentsgiven
      
      
  
  



Test cases

# case 1

Let the regression equation be given by

$y = 0 + 0*cov1 + 2*expt1 - 4*expt1*cov1$

One can imagine that $cov1$ is gender e.g. $cov1 == 1$ means male user, while $cov1 == 0$ indicates female user.

And $expt1 == 1$ indicates the user is shown a male picture ($expt1 == 0$ indicates a female picture).

From the regression equation, one can see that the optimal policy is to show males pictures of females, and vice-versa.


# case 2

$y = 0 + 0*cov1 + 5*expt1 -10*expt1*cov1 -5*expt2 + 10*expt2*cov1$

This is similar to case 1, but there are two experimental treatments instead of 1.

Optimal policy:

if cov1 == 0: expt1 = 1 and expt2 = 0

if cov1 == 1: expt1 = 0 and expt2 = 1

# case 3

$y = 4 + 3*cov1 - 2*cov2 + 20*expt1 -14*expt1*cov1 -14*expt1*cov2$

In this case, there are two covariates and one experimental treatment.

Optimal Policy is for expt1 to be 0 only when both cov1 and cov2 are zero, otherwise expt1 should be 1

# case 4
$y = 4 + 3*cov1 -2*cov2 -8*expt1 +12*expt1*cov1 + 11*expt1*cov2 -9*expt2 + 6*expt2*cov1 + 7*expt2*cov2$

There are two covariates and two experimental treatments in this case.

Optimal policy:
expt1: should = 1 when cov1 OR cov2 = 1
expt2: should = 1 when cov1 AND cov2 = 1



In [0]:
import pandas as pd

def printchosenaction(startobs, endobs):
  category = datasource[startobs - 1:endobs]
  chosenaction = treatmentschosen[:, startobs - 1:endobs]
  df = pd.DataFrame(
      {
          "obs no.": [i for i in range(startobs, endobs + 1)],
      }
  
  )
  for i in range(1, numcovar + 1):
    df["cov" + str(i)] = [j[i-1] for j in category]
    
  for i in range(1, numexptvar + 1):
    df["expt" + str(i)] = [int(j) for j in chosenaction[i-1]]
  print(df[["obs no."] + ["cov" + str(i) for i in range(1, numcovar + 1)] + ["expt" + str(i) for i in range(1, numexptvar + 1)]])
  
  
# case 1
# [constant cov1 expt1 expt1*cov1]
# if cov1 == 0: !expt1
# if cov1 == 1: expt1
numcovar = 1
numexptvar = 1
numregcoef = (numcovar + 1)*(numexptvar + 1) # 2*2 = 4
numobs = 500
coefs = [0, 0, 2, -4]

linbayes = LinearBayes(a_m0 = np.zeros(numregcoef), m_S0 = 100*np.eye(numregcoef), beta = 0.1)
datasource = np.array([[np.random.randint(2)] for _ in range(numobs)])
expt = Experiment(numcovar, numexptvar, linbayes, datasource, coefs)
treatmentschosen = expt.run_experiment(numobs)

print("Case 1: \n") 
print("expt1's value indicates which experimental treatment that the contextual multi-armed bandit assigned. \n") 

print("Initially, the bandit algorithm makes mistakes when assigning users to treatments. See below table for the first ten users. \n")

print("Recall that cov1 == 1 could denote male users, cov1 == 0 indicates female users. expt1 == 1 denotes male photos while expt1 == 0 indicates female photos. \n")

print("Notice the system makes mistakes: Some users see photos of the same gender, even though the optimal policy is to assign photos of the opposite gender. \n")


printchosenaction(1, 10)

print("\n")

print("The below table shows the treatments given to users 491 to 500. \n")

print("Observe that the system has learnt the optimal personalized policy: male users are shown female photos, and vice-versa. \n")

printchosenaction(491, 500)



Case 1: 

expt1's value indicates which experimental treatment that the contextual multi-armed bandit assigned. 

Initially, the bandit algorithm makes mistakes when assigning users to treatments. See below table for the first ten users. 

Recall that cov1 == 1 could denote male users, cov1 == 0 indicates female users. expt1 == 1 denotes male photos while expt1 == 0 indicates female photos. 

Notice the system makes mistakes: Some users see photos of the same gender, even though the optimal policy is to assign photos of the opposite gender. 

   obs no.  cov1  expt1
0        1     0      1
1        2     1      1
2        3     1      1
3        4     0      1
4        5     1      0
5        6     1      0
6        7     0      1
7        8     0      0
8        9     0      1
9       10     1      0


The below table shows the treatments given to users 491 to 500. 

Observe that the system has learnt the optimal personalized policy: male users are shown female photos, and vice-versa.

In [0]:
# case 2
# [constant cov1 expt1 expt1*cov1 expt2 expt2*cov1]
# if cov1 == 0: expt1 and !expt2
# if cov1 == 1: !expt1 and ~expt2
numcovar = 1
numexptvar = 2
numregcoef = (numcovar + 1)*(numexptvar + 1) # 2*3 = 6
numobs = 500
coefs = [0, 0, 5, -10, -5, 10]

linbayes = LinearBayes(a_m0 = np.zeros(numregcoef), m_S0 = 100*np.eye(numregcoef), beta = 0.1)
datasource = np.array([[np.random.randint(2)] for _ in range(numobs)])
expt = Experiment(numcovar, numexptvar, linbayes, datasource, coefs)
treatmentschosen = expt.run_experiment(numobs)

print("Case 2: \n") 

print("Again, the system makes mistakes initially but learns the optimal personalized policy as users make decisions. \n")
printchosenaction(1, 10)

print("\n")

printchosenaction(491, 500)


Case 2: 

Again, the system makes mistakes initially but learns the optimal personalized policy as users make decisions. 

   obs no.  cov1  expt1  expt2
0        1     0      1      0
1        2     0      1      1
2        3     0      1      1
3        4     0      1      0
4        5     0      1      0
5        6     1      0      1
6        7     1      0      0
7        8     1      0      1
8        9     0      0      0
9       10     1      1      0


   obs no.  cov1  expt1  expt2
0      491     1      0      1
1      492     1      0      1
2      493     1      0      1
3      494     0      1      0
4      495     1      0      1
5      496     0      1      0
6      497     0      1      0
7      498     1      1      1
8      499     0      1      0
9      500     1      0      1


In [0]:
# case 3
# [constant cov1 cov2 expt1 expt1*cov1 expt1*cov2]
# expt1 should = 0 iff cov1 and cov2 are 1
numcovar = 2
numexptvar = 1
numregcoef = (numcovar + 1)*(numexptvar + 1) # 2*3 = 6
numobs = 500
coefs = [4, 3, -2, 20, -14, -14]

linbayes = LinearBayes(a_m0 = np.zeros(numregcoef), m_S0 = 100*np.eye(numregcoef), beta = 0.1)
datasource = np.array([[np.random.randint(2), np.random.randint(2)] for _ in range(numobs)])
expt = Experiment(numcovar, numexptvar, linbayes, datasource, coefs)
treatmentschosen = expt.run_experiment(numobs)

print("Case 3: \n") 
printchosenaction(1, 10)
printchosenaction(491, 500)


Case 3: 

   obs no.  cov1  cov2  expt1
0        1     0     1      1
1        2     1     1      0
2        3     0     0      1
3        4     1     1      1
4        5     0     1      0
5        6     1     0      0
6        7     0     0      1
7        8     0     0      1
8        9     0     0      1
9       10     0     0      1
   obs no.  cov1  cov2  expt1
0      491     1     1      0
1      492     0     1      1
2      493     1     1      0
3      494     1     1      0
4      495     0     0      1
5      496     0     0      1
6      497     0     0      1
7      498     1     0      1
8      499     0     0      1
9      500     1     1      0


In [0]:
# case 4
# [constant cov1 cov2 expt1 expt1*cov1 expt1*cov2 expt2 expt2*cov1 expt2*cov2]
# expt1: OR
# expt2: AND

numcovar = 2
numexptvar = 2
numregcoef = (numcovar + 1)*(numexptvar + 1) # 3*3 = 9
numobs = 500
coefs = [4, 3, -2, -8, 12, 11, -9, 6, 7]

linbayes = LinearBayes(a_m0 = np.zeros(numregcoef), m_S0 = 100*np.eye(numregcoef), beta = 0.1)
datasource = np.array([[np.random.randint(2), np.random.randint(2)] for _ in range(numobs)])
expt = Experiment(numcovar, numexptvar, linbayes, datasource, coefs)
treatmentschosen = expt.run_experiment(numobs)

print("Case 4: \n") 
printchosenaction(1, 10)
printchosenaction(491, 500)


Case 4: 

   obs no.  cov1  cov2  expt1  expt2
0        1     0     0      0      1
1        2     0     1      0      1
2        3     1     1      0      0
3        4     1     0      1      1
4        5     0     1      1      0
5        6     1     1      0      0
6        7     0     1      1      1
7        8     1     0      1      0
8        9     1     0      0      0
9       10     0     0      1      1
   obs no.  cov1  cov2  expt1  expt2
0      491     0     1      1      0
1      492     1     0      1      0
2      493     0     1      1      0
3      494     0     1      1      0
4      495     1     1      1      1
5      496     0     0      0      0
6      497     0     1      1      0
7      498     1     0      1      0
8      499     1     1      1      1
9      500     1     0      1      0
